In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [12]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [13]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')
tables = soup.find_all('table',class_='wikitable sortable')

In [18]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [23]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [27]:
df_2021 = df[['Title','Cast and crew']]

In [28]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '0637662c8bb31d6ff455570552952b85'

In [29]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [31]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

In [32]:
#getting directors
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

#getting actors
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [34]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(x))
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(x))
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(x))
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(x))

In [35]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Crime Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
326,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Family Comedy Animation Drama Music,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson
327,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,War Action Thriller Adventure,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans
328,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama Thriller War,Joel Coen,Denzel Washington,Frances McDormand,Corey Hawkins
329,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid


In [36]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [37]:
data_2021 = df_2021[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [38]:
data_2021

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire
...,...,...,...,...,...,...
326,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,Sing 2
327,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,The King's Man
328,Joel Coen,Denzel Washington,Frances McDormand,Corey Hawkins,Drama Thriller War,The Tragedy of Macbeth
329,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog


In [39]:
data_2021['summary'] = data_2021['actor_1_name'] + ' ' + data_2021['actor_2_name'] + ' '+ \
data_2021['actor_3_name'] + ' '+ data_2021['director_name'] +' ' + data_2021['genres']

In [40]:
data_2021.isnull().sum()

director_name     0
actor_1_name      0
actor_2_name      7
actor_3_name     21
genres            0
movie_title       0
summary          21
dtype: int64

In [41]:
data_2021.dropna(how='any', inplace=True)

In [43]:
data_2021['movie_title'] = data_2021['movie_title'].str.lower()

In [44]:
data_2021

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,summary
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
326,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
327,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
328,Joel Coen,Denzel Washington,Frances McDormand,Corey Hawkins,Drama Thriller War,the tragedy of macbeth,Denzel Washington Frances McDormand Corey Hawk...
329,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [46]:
data_2021.to_csv('processed_datasets/data_2021.csv')

handle: processed_datasets/data_2021.csv
processed_datasets/data_2021.csv


### Merging 2021 to 2016-2020

In [45]:
data_16_20 = pd.read_csv('processed_datasets/data_16_20.csv')

handle: processed_datasets/data_16_20.csv
processed_datasets/data_16_20.csv


In [47]:
data_16_21 = data_16_20.append(data_2021, ignore_index=False)

In [48]:
data_16_21

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,summary
0,James Cameron,Joel David Moore,Action Adventure Fantasy Sci-Fi,CCH Pounder,avatar,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Orlando Bloom,Action Adventure Fantasy,Johnny Depp,pirates of the caribbean: at world's end,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Rory Kinnear,Action Adventure Thriller,Christoph Waltz,spectre,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Christian Bale,Action Thriller,Tom Hardy,the dark knight rises,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Rob Walker,Documentary,Doug Walker,star wars: episode vii - the force awakens,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
326,Garth Jennings,Reese Witherspoon,Family Comedy Animation Drama Music,Matthew McConaughey,sing 2,Scarlett Johansson,Matthew McConaughey Reese Witherspoon Scarlett...
327,Matthew Vaughn,Gemma Arterton,War Action Thriller Adventure,Ralph Fiennes,the king's man,Rhys Ifans,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
328,Joel Coen,Frances McDormand,Drama Thriller War,Denzel Washington,the tragedy of macbeth,Corey Hawkins,Denzel Washington Frances McDormand Corey Hawk...
329,Erwin brothers,Anna Paquin,Drama,Zachary Levi,american underdog,Dennis Quaid,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [49]:
data_16_21.to_csv('processed_datasets/data_16_21.csv')

handle: processed_datasets/data_16_21.csv
processed_datasets/data_16_21.csv
